In [1]:
pip install xlrd

In [2]:
import pandas as pd

In [3]:
#Puxando as bases de analise SAP e WMS e renomenado as colunas do 819
wms_df = pd.read_excel(r'C:\Users\lucasr.silva\Desktop\Gallo\Confronto WMS X SAP\Semana 48\819 GALLO.xls')
sap_df = pd.read_excel(r'C:\Users\lucasr.silva\Desktop\Gallo\Confronto WMS X SAP\Semana 48\SAP GALLO.xlsx')
column_names = {
    'Praz Min': 'C.Custo',
    'Praz Min G.R.': 'Cliente',
    'OK\nPraz Min ': 'Produto',
    'Ok? Praz Min G.R.': 'Descrição',
    'Giro': 'UN',
    'Bloq End.': 'Vencto.',
    'End.\nPick.': 'Lote',
    'Grupo': 'Nota Entr.',
    'Unnamed: 8': 'Data Entr. NF',
    'Unnamed: 9': 'Tipo',
    'Unnamed: 10': 'UU',
    'Unnamed: 11': 'Dep',
    'Unnamed: 12': 'CD',
    'Unnamed: 13': 'Setor',
    'Unnamed: 14': 'SubSetor',
    'Unnamed: 15': 'Rua',
    'Unnamed: 16': 'Predio',
    'Unnamed: 17': 'Andar',
    'Unnamed: 18': 'Qtd',
    'Unnamed: 19': 'Reserva',
    'Unnamed: 20': 'Vl. Unit',
    'Unnamed: 21': 'Shelf Life?',
    'Unnamed: 22': 'Praz Min',
    'Unnamed: 23': 'Praz Min G.R.',
    'Unnamed: 24': 'OK',
    'Unnamed: 25': 'Ok? Praz Min G.R.',
    'Unnamed: 26': 'Giro',
    'Unnamed: 27': 'Bloq End.',
    'Unnamed: 28': 'End.',
    'Unnamed: 29': 'Status Endereço',
    'Unnamed: 30': 'Bloqueio Gestão',
    'Unnamed: 31': 'Grupo',
}

wms_df = wms_df.rename(columns=column_names)

In [4]:
#Filtrando as colunas da planilhas de acordo com a analise
wms_df = wms_df[['Produto','Descrição','Lote','Vencto.','Status Endereço']]
sap_df = sap_df[['Material','Batch','Shelf life exp. date']]

wms_df['Produto'] = wms_df['Produto'].astype(str)
wms_df['Produto'] = wms_df['Produto'].str.rstrip(".0")

In [5]:
#concatenando as colunas de dataas e ID
wms_df['Mês WMS'] = wms_df['Vencto.'].dt.month
wms_df['Ano WMS'] = wms_df['Vencto.'].dt.year
wms_df['ID'] = wms_df['Produto'].astype(str) + ' ' + wms_df['Lote'].astype(str)
wms_df['Mês/Ano WMS'] = wms_df['Mês WMS'].astype(str) + '/' + wms_df['Ano WMS'].astype(str)

sap_df['Mês SAP'] = sap_df['Shelf life exp. date'].dt.month
sap_df['Ano SAP'] = sap_df['Shelf life exp. date'].dt.year
sap_df['ID'] = sap_df['Material'].astype(str) + ' ' + sap_df['Batch'].astype(str)
sap_df['Mês/Ano SAP'] = sap_df['Mês SAP'].astype(str) + '/' + sap_df['Ano SAP'].astype(str)

In [6]:
#Merge de SAP com WMS
confronto_df = sap_df.merge(wms_df,on='ID',how='inner')

In [7]:
#Trazendo as colunas desejadas do confronto
confronto_df = confronto_df[['Produto','Descrição','Lote','ID','Shelf life exp. date','Vencto.','Mês/Ano SAP','Mês/Ano WMS','Status Endereço']]
# Supondo que 'Shelf life exp. date' e 'Vencto.' estão no formato 'YYYY-MM-DD'
confronto_df['Shelf life exp. date'] = pd.to_datetime(confronto_df['Shelf life exp. date'], format='%d/%m/%Y')
confronto_df['Vencto.'] = pd.to_datetime(confronto_df['Vencto.'], format='%d/%m/%Y')

confronto_df['Mês/Ano SAP'] = confronto_df['Mês/Ano SAP'].str.rstrip(".0")
confronto_df['Mês/Ano WMS'] = confronto_df['Mês/Ano WMS'].str.rstrip(".0")

In [8]:
#Trazendo as colunas desejadas do confronto adicionando colunas de formulas
def lote(vsap, vwms):
    if vsap == vwms:
        return 'Lote OK'
    else:
        return 'Lote Nok'

confronto_df['Status_Lote'] = confronto_df.apply(lambda x: lote(x['Mês/Ano SAP'], x['Mês/Ano WMS']), axis=1)

def lote(acao):
    if acao == 'Lote Nok':
        return 'Validar Fisico'
    else:
        return '-'

confronto_df['Ação'] = confronto_df['Status_Lote'].apply(lote)

In [9]:
import os

# Definindo o caminho da pasta e do arquivo
pasta = r"C:\Users\lucasr.silva\Desktop\Gallo\Confronto WMS X SAP\Semana 48"
arquivo = "confront_df.xlsx"
caminho_completo = os.path.join(pasta, arquivo)

# Criando a pasta, se não existir
os.makedirs(pasta, exist_ok=True)

# Salvando o DataFrame
try:
    confronto_df.to_excel(caminho_completo, index=False)
    print("DataFrame salvo com sucesso!")
except OSError as e:
    print(f"Erro ao salvar o arquivo: {e}")

DataFrame salvo com sucesso!
